# Web-Usage Mining

## Import Libraries

In [ ]:
import pandas as pd
import re
import json
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Collection

In [ ]:
path = '/content/drive/MyDrive/Y4_T2/Data Mining/Web-Usage Mining/'

In [ ]:
with open(path+'kaggle.json') as json_file:
    kaggle_json = json.load(json_file)

In [ ]:
kaggle_json

{'key': 'de2831679e4ee1cbc3c704c966f19f92', 'username': 'joycenjeri'}

In [ ]:
# create the .kaggle directory and an empty kaggle.json file
!mkdir -p /root/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json
kaggle_username = "joycenjeri"
kaggle_key = "de2831679e4ee1cbc3c704c966f19f92"

# Save API token the kaggle.json file
with open("/root/.kaggle/kaggle.json", "w") as f:
    f.write(json.dumps({"username": kaggle_username, "key": kaggle_key}))

In [ ]:
!kaggle datasets download -d eliasdabbas/web-server-access-logs

web-server-access-logs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip -o web-server-access-logs.zip

Archive:  web-server-access-logs.zip
  inflating: access.log              
  inflating: client_hostname.csv     


In [ ]:
!head -n 5 access.log

54.36.149.41 - - [22/Jan/2019:03:56:14 +0330] "GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,27|%DA%A9%D9%85%D8%AA%D8%B1%20%D8%A7%D8%B2%205%20%D9%85%DA%AF%D8%A7%D9%BE%DB%8C%DA%A9%D8%B3%D9%84,p53 HTTP/1.1" 200 30577 "-" "Mozilla/5.0 (compatible; AhrefsBot/6.1; +http://ahrefs.com/robot/)" "-"
31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/60844/productModel/200x200 HTTP/1.1" 200 5667 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"
31.56.96.51 - - [22/Jan/2019:03:56:16 +0330] "GET /image/61474/productModel/200x200 HTTP/1.1" 200 5379 "https://www.zanbil.ir/m/filter/b113" "Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build/HuaweiALE-L21) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.158 Mobile Safari/537.36" "-"
40.77.167.129 - - [22/Jan/2019:03:56:17 +0330] "GET /image/14925/productModel/100x100 HTTP/1.1" 200 1696 

In [ ]:
!tail -n 5 access.log

188.229.21.56 - - [26/Jan/2019:20:29:13 +0330] "GET /content/view/shoppingRules HTTP/1.1" 302 0 "https://www.zanbil.ir/m/product/32106/62372/%D9%85%D8%A7%D8%B4%DB%8C%D9%86-%D9%84%D8%A8%D8%A7%D8%B3%D8%B4%D9%88%DB%8C%DB%8C-%D8%AF%D8%B1%D8%A8-%D8%A7%D8%B2-%D8%AC%D9%84%D9%88-%D8%A7%D9%84-%D8%AC%DB%8C-%D9%85%D8%AF%D9%84-WM-865CW" "Mozilla/5.0 (Linux; Android 7.0; SM-N920C Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Mobile Safari/537.36" "-"
5.127.220.71 - - [26/Jan/2019:20:29:13 +0330] "GET /apple-touch-icon-120x120.png HTTP/1.1" 404 32420 "-" "MobileSafari/604.1 CFNetwork/976 Darwin/18.2.0" "-"
5.213.7.50 - - [26/Jan/2019:20:29:13 +0330] "GET /m/product/18962/%D8%BA%D8%B0%D8%A7-%D8%B3%D8%A7%D8%B2-%D9%85%D9%88%D9%84%DB%8C%D9%86%DA%A9%D8%B3-%D9%85%D8%AF%D9%84-FP7367RT HTTP/1.1" 200 20959 "https://www.google.com/" "Mozilla/5.0 (iPhone; CPU iPhone OS 10_2_1 like Mac OS X) AppleWebKit/602.4.6 (KHTML, like Gecko) Version/10.0 Mobile/14D27 Safari/602.1" "-"
109.125.16

In [ ]:
# data entries
with open('access.log') as log:
  lines = len(log.readlines())
  print(lines)

10365152


In [ ]:
arr = []
# source: https://www.oreilly.com/library/view/regular-expressions-cookbook/9781449327453/ch07s12.html
pattern = re.compile(r'^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-) "(?P<referrer>[^"]*)" "(?P<useragent>[^"]*)')
with open('access.log') as log:
  for i in range(0, lines):
    line = log.readline()
    match = re.search(pattern, line)
    if (match):
      arr.append([match.group(1), match.group(3), match.group(4)+' '+match.group(5), match.group(6), match.group(7), match.group(8), match.group(9)])

In [ ]:
data = pd.DataFrame(arr, columns=['Client_IP_address', 'Time_of_activity', 'Requested_Page', 'Status_Code', 'Size_of_Page',	'Referer_page',	'Client_Device'])

In [ ]:
data.head()

,Client_IP_address,Time_of_activity,Requested_Page,Status_Code,Size_of_Page,Referer_page,Client_Device
0,54.36.149.41,22/Jan/2019:03:56:14 +0330,GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%D...,200,30577,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...
1,31.56.96.51,22/Jan/2019:03:56:16 +0330,GET /image/60844/productModel/200x200,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
2,31.56.96.51,22/Jan/2019:03:56:16 +0330,GET /image/61474/productModel/200x200,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
3,40.77.167.129,22/Jan/2019:03:56:17 +0330,GET /image/14925/productModel/100x100,200,1696,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
4,91.99.72.15,22/Jan/2019:03:56:17 +0330,GET /product/31893/62100/%D8%B3%D8%B4%D9%88%D8...,200,41483,-,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...


In [ ]:
data.tail()

,Client_IP_address,Time_of_activity,Requested_Page,Status_Code,Size_of_Page,Referer_page,Client_Device
10364860,188.229.21.56,26/Jan/2019:20:29:13 +0330,GET /content/view/shoppingRules,302,0,https://www.zanbil.ir/m/product/32106/62372/%D...,Mozilla/5.0 (Linux; Android 7.0; SM-N920C Buil...
10364861,5.127.220.71,26/Jan/2019:20:29:13 +0330,GET /apple-touch-icon-120x120.png,404,32420,-,MobileSafari/604.1 CFNetwork/976 Darwin/18.2.0
10364862,5.213.7.50,26/Jan/2019:20:29:13 +0330,GET /m/product/18962/%D8%BA%D8%B0%D8%A7-%D8%B3...,200,20959,https://www.google.com/,Mozilla/5.0 (iPhone; CPU iPhone OS 10_2_1 like...
10364863,109.125.169.52,26/Jan/2019:20:29:13 +0330,GET /image/%7B%7BbasketItem.id%7D%7D?type=prod...,200,5,https://www.zanbil.ir/,Mozilla/5.0 (Windows NT 6.1; rv:64.0) Gecko/20...
10364864,37.129.59.160,26/Jan/2019:20:29:13 +0330,GET /basket/view,200,17299,https://www-zanbil-ir.cdn.ampproject.org/v/s/w...,Mozilla/5.0 (Linux; Android 6.0.1; D6633 Build...


In [ ]:
data.shape

(10364865, 7)

## Data Preprocessing

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364865 entries, 0 to 10364864
Data columns (total 7 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   Client_IP_address  object
 1   Time_of_activity   object
 2   Requested_Page     object
 3   Status_Code        object
 4   Size_of_Page       object
 5   Referer_page       object
 6   Client_Device      object
dtypes: object(7)
memory usage: 553.5+ MB


In [ ]:
data.Time_of_activity = data.Time_of_activity.str[:-6]

In [ ]:
data.Time_of_activity = data.Time_of_activity.apply(lambda x: x.replace('Jan', '01'))

In [ ]:
data.Time_of_activity = pd.to_datetime(data.Time_of_activity, format='%d/%m/%Y:%H:%M:%S')

In [ ]:
data.Status_Code = data.Status_Code.astype(int)

In [ ]:
data.Size_of_Page = data.Size_of_Page.astype(int)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10364865 entries, 0 to 10364864
Data columns (total 7 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Client_IP_address  object        
 1   Time_of_activity   datetime64[ns]
 2   Requested_Page     object        
 3   Status_Code        int64         
 4   Size_of_Page       int64         
 5   Referer_page       object        
 6   Client_Device      object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 553.5+ MB


In [ ]:
# save to drive
path = '/content/drive/MyDrive/Y4_T2/Data Mining/Web-Usage Mining/server_logs.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  data.to_csv(f)